<a href="https://colab.research.google.com/github/krausnick/BigData-Gruppe-3/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Um eine logistische Regression durchführen zu können, müssen wir die nötige Library "sklearn" laden. Des Weiteren benötigen wir für die Datenverarbeitung, die Library "pandas".

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Einlesen des Datensatzes von Kaggle
Quelle: https://www.kaggle.com/datasets/ealaxi/paysim1


In [ ]:
df = pd.read_csv('Fraud.csv')

Data Understanding: Datensatz wurde importiert und in der Variable "df" gespeichert. Anschließend im Rahmen des Data Understanding, wird die Funktion "df.desribe()" aufgerufen um ein Datenverständis zu realisieren und um mögliche Aureißer zu finden.

In [ ]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,151799.000000,1.517990e+05,1.517990e+05,1.517980e+05,1.517980e+05,1.517980e+05,151798.000000,151798.0
mean,9.333902,1.791774e+05,8.879109e+05,9.043402e+05,9.137641e+05,1.180076e+06,0.000863,0.0
std,1.906281,3.401606e+05,2.807531e+06,2.844268e+06,2.384389e+06,2.705635e+06,0.029364,0.0
min,1.000000,3.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,9.000000,1.111398e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,10.000000,6.122687e+04,1.999000e+04,0.000000e+00,3.164400e+04,9.332977e+04,0.000000,0.0
75%,11.000000,2.230398e+05,1.956685e+05,2.262719e+05,6.869059e+05,1.120989e+06,0.000000,0.0
max,12.000000,1.000000e+07,3.893942e+07,3.894623e+07,3.490552e+07,3.894623e+07,1.000000,0.0


In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


Im ursprünglichen Datensatz gab es nur ca 130 Fraud-Fälle. Deshalb haben wir uns dazu entschieden synthetisch mehr Fraud-Fälle zu erstellen und diese an den originalen Datensatz anzuhängen. Dadurch kann unser Modell besser aus den Daten lernen. Wir haben uns für den Parameter 1000 entschieden um eine ungefähr gleiches Verhältnis zwischen Fraud- und Nicht-Fraud-Fällen im Datensatz herzustellen. Außerdem haben wir noch die Spalten "nameOrig" & "nameDest" eliminiert, da diese Werte enthielten die nicht mit unserem Modell verwertert werden konnten und auch nicht durch ein One-Hot-Encoding binär umkodiert werden konnten, da es zu viele waren. Zudem haben wir noch NA-Werte aus dem Datensatz eliminiert.

In [ ]:
df = df.drop('type', axis=1)
new_df = df[df['isFraud'] == 1]
df_list = [new_df] * 1000
repeated_df = pd.concat(df_list, ignore_index=True)

df= pd.concat([repeated_df, df], ignore_index=True)

X = df.drop('isFraud', axis=1)
X= X.drop('nameOrig', axis=1)
X= X.drop('nameDest',axis=1)
X.dropna(axis=0, inplace=True)
df= df.drop('nameOrig', axis=1)
df= df.drop('nameDest',axis=1)
df.dropna(axis=0, inplace=True)

count = df['isFraud'].value_counts()
print(count)
y = df['isFraud']

0.0    151667
1.0    131131
Name: isFraud, dtype: int64


Einteilung der Daten in Trainings- und Testdaten. Im Verhältnis 92:8. Wir haben einen niedrigeren Wert als die üblichen 80:20 verwendet, da unser Datensatz sehr viele Datenpunkte enthält.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.08, random_state=42)


Logistisches Regressionsmodell wird trainiert mit 1000000 Interations, dies haben wir gemacht, da mit der Standarteinstellung, ein Python-Fehler ausgegeben wurde.

In [ ]:
model = LogisticRegression(max_iter=1000000)

model.fit(X_train, y_train)


LogisticRegression(max_iter=1000000)

Hier berechnen wir das logistische Regressionsmodell auf die Testdaten

In [ ]:
y_pred = model.predict(X_test)


Hier Bewerten wir unser Modell mit folgenden Metriken:
- Genauigkeit
- ConfusionMatrix
- F1-Score

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Genauigkeit des Modells:", accuracy)
print("Verwirrungsmatrix:\n", confusion)
print("Klassifikationsbericht:\n", report)

Genauigkeit des Modells: 0.9080180339462518
Verwirrungsmatrix:
 [[10955  1291]
 [  790  9588]]
Klassifikationsbericht:
               precision    recall  f1-score   support

         0.0       0.93      0.89      0.91     12246
         1.0       0.88      0.92      0.90     10378

    accuracy                           0.91     22624
   macro avg       0.91      0.91      0.91     22624
weighted avg       0.91      0.91      0.91     22624



Alles in allem, hat das Modell einen F1-Score von ca. 90% für die relevante Variable "isFraud" erreicht und somit unsere voherig festgelegten Anforderungen aus dem Miro-Board erfüllt.